In [1]:
import argparse
import time
import shutil
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import os.path as osp
import csv
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, MultiStepLR
from model import SGN
from data import NTUDataLoaders, AverageMeter
from util import make_dir, get_num_classes
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import random

np.random.seed(42)

In [2]:
X = pickle.load(open('data/X.pkl', 'rb'))

In [3]:
samples = 5000
train_videos = {}
test_videos = {}

for video in X.keys():
    actor = int(video[9:12])
    action = int(video[17:20])
    
    if len(X[video]) == 0: continue
    if action > 60:
        if actor not in test_videos:
            test_videos[actor] = []
        test_videos[actor].append(X[video])    
    else:
        if actor not in train_videos:
            train_videos[actor] = []
        train_videos[actor].append(X[video])

In [4]:
train_x = np.zeros((samples*2, 300, 150), dtype=np.float32)
val_x = np.zeros((samples*2, 300, 150), dtype=np.float32)
test_x = np.zeros((samples*2, 300, 150), dtype=np.float32)

train_y = np.concatenate((np.ones(samples), np.zeros(samples)))
val_y = np.concatenate((np.ones(samples), np.zeros(samples)))
test_y = np.concatenate((np.ones(samples), np.zeros(samples)))

train_y = np.eye(2)[train_y.astype('int32')]
val_y = np.eye(2)[val_y.astype('int32')]
test_y = np.eye(2)[test_y.astype('int32')]

# Same
for i in range(samples):
    actor = random.choice(list(train_videos.keys()))
    vid1 = random.choice(train_videos[actor])
    vid2 = random.choice(train_videos[actor])
    train_x[i] = np.concatenate((vid1, vid2), axis=1)

    actor = random.choice(list(test_videos.keys()))
    vid1 = random.choice(test_videos[actor])
    vid2 = random.choice(test_videos[actor])
    val_x[i] = np.concatenate((vid1, vid2), axis=1)

    actor = random.choice(list(test_videos.keys()))
    vid1 = random.choice(test_videos[actor])
    vid2 = random.choice(test_videos[actor])
    test_x[i] = np.concatenate((vid1, vid2), axis=1)
    

# Diff
for i in range(samples):
    actor = random.choice(list(train_videos.keys()))
    vid1 = random.choice(train_videos[actor])
    actor2 = random.choice(list(train_videos.keys()))
    while actor != actor2:
        actor2 = random.choice(list(train_videos.keys()))
    vid2 = random.choice(train_videos[actor2])
    train_x[i+samples] = np.concatenate((vid1, vid2), axis=1)   

    actor = random.choice(list(test_videos.keys()))
    vid1 = random.choice(test_videos[actor])
    actor2 = random.choice(list(test_videos.keys()))
    while actor != actor2:
        actor2 = random.choice(list(test_videos.keys()))
    vid2 = random.choice(test_videos[actor2])
    val_x[i+samples] = np.concatenate((vid1, vid2), axis=1)

    actor = random.choice(list(test_videos.keys()))
    vid1 = random.choice(test_videos[actor])
    actor2 = random.choice(list(test_videos.keys()))
    while actor != actor2:
        actor2 = random.choice(list(test_videos.keys()))
    vid2 = random.choice(test_videos[actor2])
    test_x[i+samples] = np.concatenate((vid1, vid2), axis=1)

# SGN

In [5]:
import time
import shutil
import os
import os.path as osp
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from model2 import SGN
from data import NTUDataLoaders, AverageMeter
from util import make_dir, get_num_classes

# Hyperparameters/Tuning Parameters
network='SGN'
dataset='NTU'
start_epoch=0
case=0
batch_size=64
max_epochs=500
monitor='val_acc'
lr=0.001
weight_decay=0.0001
lr_factor=0.1
workers=16
print_freq = 20
do_train=1
seg=20

def train(train_loader, model, criterion, optimizer, epoch):
    losses = AverageMeter()
    acces = AverageMeter()
    model.train()

    for i, (inputs, target) in enumerate(train_loader):

        output = model(inputs.cuda())
        target = target.cuda()
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc = accuracy(output.data, target)
        losses.update(loss.item(), inputs.size(0))
        acces.update(acc[0], inputs.size(0))

        # backward
        optimizer.zero_grad()  # clear gradients out before each mini-batch
        loss.backward()
        optimizer.step()

        if (i + 1) % print_freq == 0:
            print('Epoch-{:<3d} {:3d} batches\t'
                  'loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'accu {acc.val:.3f} ({acc.avg:.3f})'.format(
                      epoch + 1, i + 1, loss=losses, acc=acces))

    return losses.avg, acces.avg


def validate(val_loader, model, criterion):
    losses = AverageMeter()
    acces = AverageMeter()
    model.eval()

    for i, (inputs, target) in enumerate(val_loader):
        with torch.no_grad():
            output = model(inputs.cuda())
        target = target.cuda()
        with torch.no_grad():
            loss = criterion(output, target)

        # measure accuracy and record loss
        acc = accuracy(output.data, target)
        losses.update(loss.item(), inputs.size(0))
        acces.update(acc[0], inputs.size(0))

    return losses.avg, acces.avg


def test(test_loader, model, checkpoint, lable_path, pred_path):
    acces = AverageMeter()
    # load learnt model that obtained best performance on validation set
    model.load_state_dict(torch.load(checkpoint)['state_dict'])
    model.eval()

    label_output = list()
    pred_output = list()

    t_start = time.time()
    for i, t in enumerate(test_loader):
        inputs = t[0]
        target = t[1]
        with torch.no_grad():
            output = model(inputs.cuda())
            output = output.view(
                (-1, inputs.size(0)//target.size(0), output.size(1)))
            output = output.mean(1)

        label_output.append(target.cpu().numpy())
        pred_output.append(output.cpu().numpy())

        acc = accuracy(output.data, target.cuda())
        acces.update(acc[0], inputs.size(0))

    label_output = np.concatenate(label_output, axis=0)
    np.savetxt(lable_path, label_output, fmt='%d')
    pred_output = np.concatenate(pred_output, axis=0)
    np.savetxt(pred_path, pred_output, fmt='%f')

    print('Test: accuracy {:.3f}, time: {:.2f}s'
          .format(acces.avg, time.time() - t_start))


def accuracy(output, target):
    batch_size = target.size(0)
    _, pred = output.topk(1, 1, True, True)
    pred = pred.t()
    target = torch.argmax(target, dim=1)  # Add this line to convert one-hot targets to class indices
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    correct = correct.view(-1).float().sum(0, keepdim=True)
    return correct.mul_(100.0 / batch_size)



def save_checkpoint(state, filename='checkpoint.pth.tar', is_best=False):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


def get_n_params(model):
    pp = 0
    for p in list(model.parameters()):
        nn = 1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp


class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            target = torch.argmax(target, dim=1)  # Add this line to convert one-hot targets to class indices
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))


In [6]:
def main():
    num_classes = get_num_classes(dataset)
    model = SGN(num_classes, dataset, seg, batch_size, do_train)

    total = get_n_params(model)
    # print(model)
    print('The number of parameters: ', total)
    print('The modes is:', network)

    if torch.cuda.is_available():
        print('It is using GPU!')
        model = model.cuda()

    criterion = LabelSmoothingLoss(num_classes, smoothing=0.1).cuda()
    optimizer = optim.Adam(model.parameters(), lr=lr,
                           weight_decay=weight_decay)

    if monitor == 'val_acc':
        mode = 'max'
        monitor_op = np.greater
        best = -np.Inf
        str_op = 'improve'
    elif monitor == 'val_loss':
        mode = 'min'
        monitor_op = np.less
        best = np.Inf
        str_op = 'reduce'

    scheduler = MultiStepLR(optimizer, milestones=[60, 90, 110], gamma=0.1)
    # Data loading
    ntu_loaders = NTUDataLoaders(dataset, case, seg=seg, train_X=train_x, train_Y=train_y, test_X=test_x, test_Y=test_y, val_X=val_x, val_Y=val_y, aug=0)
    train_loader = ntu_loaders.get_train_loader(batch_size, workers)
    val_loader = ntu_loaders.get_val_loader(batch_size, workers)
    train_size = ntu_loaders.get_train_size()
    val_size = ntu_loaders.get_val_size()

    test_loader = ntu_loaders.get_test_loader(32, workers)

    print('Train on %d samples, validate on %d samples' %
          (train_size, val_size))

    best_epoch = 0
    output_dir = make_dir(dataset)

    save_path = os.path.join(output_dir, network)
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    checkpoint = osp.join(save_path, '%s_best.pth' % case)
    earlystop_cnt = 0
    csv_file = osp.join(save_path, '%s_log.csv' % case)
    log_res = list()

    lable_path = osp.join(save_path, '%s_lable.txt' % case)
    pred_path = osp.join(save_path, '%s_pred.txt' % case)

    # Training
    if do_train == 1:
        for epoch in range(start_epoch, max_epochs):

            print(epoch, optimizer.param_groups[0]['lr'])

            t_start = time.time()
            train_loss, train_acc = train(
                train_loader, model, criterion, optimizer, epoch)
            val_loss, val_acc = validate(val_loader, model, criterion)
            log_res += [[train_loss, train_acc.cpu().numpy(),
                         val_loss, val_acc.cpu().numpy()]]

            print('Epoch-{:<3d} {:.1f}s\t'
                  'Train: loss {:.4f}\taccu {:.4f}\tValid: loss {:.4f}\taccu {:.4f}'
                  .format(epoch + 1, time.time() - t_start, train_loss, train_acc, val_loss, val_acc))

            current = val_loss if mode == 'min' else val_acc

            # store tensor in cpu
            current = current.cpu()

            if monitor_op(current, best):
                print('Epoch %d: %s %sd from %.4f to %.4f, '
                      'saving model to %s'
                      % (epoch + 1, monitor, str_op, best, current, checkpoint))
                best = current
                best_epoch = epoch + 1
                save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': model.state_dict(),
                    'best': best,
                    'monitor': monitor,
                    'optimizer': optimizer.state_dict(),
                }, checkpoint)
                earlystop_cnt = 0
            else:
                print('Epoch %d: %s did not %s' % (epoch + 1, monitor, str_op))
                earlystop_cnt += 1

            scheduler.step()

        print('Best %s: %.4f from epoch-%d' % (monitor, best, best_epoch))
        with open(csv_file, 'w') as fw:
            cw = csv.writer(fw)
            cw.writerow(['loss', 'acc', 'val_loss', 'val_acc'])
            cw.writerows(log_res)
        print('Save train and validation log into into %s' % csv_file)

    # Test
    model = SGN(num_classes, dataset, seg, batch_size, 0)
    model = model.cuda()
    test(test_loader, model, checkpoint, lable_path, pred_path)

main()

The number of parameters:  661294
The modes is: SGN
It is using GPU!
Train on 10000 samples, validate on 10000 samples
0 0.001
Epoch-1    20 batches	loss 0.8081 (0.8026)	accu 48.438 (49.766)
Epoch-1    40 batches	loss 0.7620 (0.7862)	accu 48.438 (49.414)
Epoch-1    60 batches	loss 0.7377 (0.7664)	accu 50.000 (49.583)
Epoch-1    80 batches	loss 0.7034 (0.7558)	accu 56.250 (50.039)
Epoch-1   100 batches	loss 0.6902 (0.7503)	accu 56.250 (49.906)
Epoch-1   120 batches	loss 0.7274 (0.7449)	accu 45.312 (49.961)
Epoch-1   140 batches	loss 0.7145 (0.7419)	accu 50.000 (49.587)
Epoch-1   49.2s	Train: loss 0.7402	accu 49.5192	Valid: loss 0.7043	accu 50.3005
Epoch 1: val_acc improved from -inf to 50.3005, saving model to ./results/NTU/SGN\0_best.pth
1 0.001
Epoch-2    20 batches	loss 0.7107 (0.7149)	accu 56.250 (51.250)
Epoch-2    40 batches	loss 0.6791 (0.7175)	accu 60.938 (49.883)
Epoch-2    60 batches	loss 0.7153 (0.7216)	accu 48.438 (49.688)
Epoch-2    80 batches	loss 0.7592 (0.7199)	accu 48.4